In [1]:
import os
os.environ['MANTLE_TARGET'] = 'ice40'
from magma import DefineCircuit, EndDefine, In, Out, Bit, wire
from mantle import DFF, LUT2, I0, I1

main = DefineCircuit('main', 'I', In(Bit), 'O', Out(Bit), 'CLK', In(Bit))

# create ff holding state first
ff = DFF()

# lut computes the next state ...
#  function of the previous state and the input
lut = LUT2( I0^I1 )
lut(main.I, ff)

# lut computes the next state
ff(lut)


wire(ff.O, main.O)
EndDefine()

print(repr(main))

import mantle lattice ice40
import mantle lattice mantle40
main = DefineCircuit("main", "I", In(Bit), "O", Out(Bit), "CLK", In(Bit))
inst0 = SB_DFF()
inst1 = SB_LUT4(LUT_INIT=0x6666)
wire(None, inst0.C)
wire(inst1.O, inst0.D)
wire(main.I, inst1.I0)
wire(inst0.Q, inst1.I1)
wire(0, inst1.I2)
wire(0, inst1.I3)
wire(inst0.Q, main.O)
EndCircuit()
